In [1]:
!pip install spacy

    100% |████████████████████████████████| 22.0MB 17kB/s  eta 0:00:01    55% |█████████████████▊              | 12.1MB 719kB/s eta 0:00:14    56% |██████████████████▏             | 12.5MB 3.1MB/s eta 0:00:04    57% |██████████████████▎             | 12.6MB 4.7MB/s eta 0:00:02
    100% |████████████████████████████████| 122kB 1.3MB/s ta 0:00:01
    100% |████████████████████████████████| 1.2MB 274kB/s ta 0:00:011
    100% |████████████████████████████████| 194kB 584kB/s ta 0:00:01
    100% |████████████████████████████████| 604kB 416kB/s ta 0:00:01    90% |████████████████████████████▉   | 542kB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 317kB 521kB/s ta 0:00:01
    100% |████████████████████████████████| 450kB 198kB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 684kB/s ta 0:00:01
  Running setup.py bdist_wheel for spacy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/60/0b/bb/7c2e28db574dbb2358176934eddd32a1c5f838ba0bc23eaaab
  Runni

In [13]:
!python -m spacy download en_core_web_sm

    100% |████████████████████████████████| 37.4MB 20.4MB/s ta 0:00:01% |                                | 71kB 59kB/s eta 0:10:25    0% |                                | 122kB 145kB/s eta 0:04:16    2% |▋                               | 757kB 434kB/s eta 0:01:25    2% |▊                               | 839kB 519kB/s eta 0:01:11    9% |███▏                            | 3.7MB 435kB/s eta 0:01:18    10% |███▎                            | 3.9MB 573kB/s eta 0:00:59    11% |███▊                            | 4.4MB 678kB/s eta 0:00:49    12% |████                            | 4.7MB 1.7MB/s eta 0:00:20    12% |████                            | 4.8MB 986kB/s eta 0:00:34    16% |█████▏                          | 6.0MB 1.5MB/s eta 0:00:22    17% |█████▊                          | 6.7MB 511kB/s eta 0:01:00    20% |██████▌                         | 7.5MB 140kB/s eta 0:03:33    22% |███████▎                        | 8.5MB 460kB/s eta 0:01:03    25% |████████▏                       | 9.5MB 533kB/s e

In [25]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 186kB/s a 0:00:01
    100% |████████████████████████████████| 829kB 68kB/s eta 0:00:011
    100% |████████████████████████████████| 71kB 231kB/s ta 0:00:011
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Sample NER Workflow for DigiVol

Read data from digiVol CSV file and pass through the SpaCY NER 

In [9]:
import spacy
import csv

We first read the data from the CSV file using the python `csv` module.  The transcript can be found in the `occurrenceRemarks` column of the file so we append the text from each row to a python list `text`.

In [6]:

text = []
with open('data/Project-1536729-DwC.csv') as fd:
    reader = csv.DictReader(fd)
    for line in reader:
        text.append(line['occurrenceRemarks'])

In [11]:
# show the first text passage
print(text[0])

Purchased this Diary for which I paid 10/ and considered reasonable, it being but 50 per Cent over the Home Cost.

	Went to the Main Gaol to hear tidings of the State Prisoners, charged with Treason in taking up arms against the Government at Balaarat.  The Judge remanded them for 10 days, stating that being charged with High Treason, they were entitled to that term of clear notice with copies of their indictments from the opening of the Sessions. Mr Ireland the Counsel for the Defence, stated he was prepared to proceed at once, but the Chief Justice preferred granting the priviledge (sic) allowed by the Law.

	This conduct seems evidently to betray an inclination on the part of the Governt. to let the matter gradually drop, and in my opinion the men will not be tried at all.

	There is a great deal of excitement in the Town, and the Gaol Authorities fearing violence have barricaded and strengthened the Entrance, as well as made a door as close to the Court as possible by which to take

In [18]:
nlp = spacy.load('en_core_web_sm')

def ner(text):

    doc = nlp(text)
    return doc.ents

In [23]:
places = []

for t in text:
    ents = ner(t)
    for ent in ents:
        if ent.label_ is "GPE":
            places.append(ent.text)
            print(ent.text)

Balaarat
Ireland
the Town
Geelong
Richmond
Melbourne
Evening
Richmond
Evening
Lotherington
Evening
Sydney
McEwen
Larder
Lotherington
Arcade
Black
Teeth
Teeth
Ingenious
Bondsmen
Asylum
Arcade
Invitation
Quarters
Query
Jersey
Arcade
Arcade
Youth
Arcade
Placards
Outbreak
Foolishly
Sandridge
Ground
Railway
Omnibus
Evening
Lotherington
Saw McEwen


In [60]:
from spacy import displacy
from IPython.core.display import display, HTML

doc = nlp(text[0])
display(HTML(displacy.render(doc, style='ent')))

Given these NER location results we will want to prune out those that are clearly wrong.  Need a manual step here to select just the locations we want to geolocate.

In [120]:
import pandas as pd

locations = pd.DataFrame({'entities': sorted(list(set(places)))})
locations

,entities
0,Arcade
1,Asylum
2,Balaarat
3,Black
4,Bondsmen
5,Evening
6,Foolishly
7,Geelong
8,Ground
9,Ingenious


In [121]:
keep = [7, 11, 12, 14, 16, 23, 24, 26]
locations = locations.iloc[keep]

In [122]:
for loc in locations['entities']:
    print(loc)

Geelong
Ireland
Jersey
Lotherington
Melbourne
Richmond
Sandridge
Sydney


In [123]:
import geocoder

geo = []
for place in locations['entities']:
    g = geocoder.osm(place)
    print(place, g.lat, g.lng, g.address)
    geo.append(g)


Geelong -38.1492813 144.3598269 Geelong, City of Greater Geelong, Barwon South West, Victoria, VIC 3220, Australia
Ireland 52.865196 -7.9794599 Ireland
Jersey 49.21230655 -2.12559995964288 Jersey
Lotherington 53.9600812 -1.0446963 Lotherington Mews, Osbaldwick, York, Yorkshire and the Humber, England, YO10 3TZ, UK
Melbourne -37.8142176 144.9631608 Melbourne, City of Melbourne, Greater Melbourne, Victoria, 3000, Australia
Richmond 37.5385087 -77.43428 Richmond, Richmond City, Virginia, 23298, United States of America
Sandridge 39.1331639 -76.7341367 Sandridge, Anne Arundel County, Maryland, 20755, United States of America
Sydney -33.8548157 151.2164539 Sydney, NSW, Australia
